# 1. 构建网络

论文名称：《Event-based High Dynamic Range Image and Very High Frame Rate Video Generation using Conditional Generative Adversarial Networks》

## 1.1 生成模型

In [1]:
import torch
import torch.nn as nn

class HDRGen(nn.Module):
    def __init__(self,dropout = 0.5):
        super(HDRGen,self).__init__()
        self.Encoder1 = nn.Sequential(
            # input(3，256，256)
            nn.Conv2d(3,64,3,stride=2,padding=1),
            nn.InstanceNorm2d(3),
            nn.ReLU()
        )
        self.Encoder2=nn.Sequential(
            #input(64,128,128)
            nn.Conv2d(64,128,3,stride=2,padding=1),
            nn.InstanceNorm2d(64),
            nn.ReLU()
        )
        self.Encoder3 = nn.Sequential(
            #input(128,64,64)
            nn.Conv2d(128,256,3,stride=2,padding=1),
            nn.InstanceNorm2d(128),
            nn.ReLU()
        )
        self.Encoder4 = nn.Sequential(
            #input(256,32,32)
            nn.Conv2d(256,512,3,stride=2,padding=1),
            nn.InstanceNorm2d(256),
            nn.ReLU()
        )
        self.Encoder5=nn.Sequential(
            #input(512,16,16)
            nn.Conv2d(512,512,3,stride=2,padding=1),
            nn.InstanceNorm2d(512),
            nn.ReLU()
        )
        self.Encoder6=nn.Sequential(
            #input(512,8,8)
            nn.Conv2d(512,512,3,stride=2,padding=1),
            nn.InstanceNorm2d(512),
            nn.ReLU()
        )
        self.Encoder7=nn.Sequential(
            #Input(512,4,4)
            nn.Conv2d(512,512,3,stride=2,padding=1),
            nn.InstanceNorm2d(512),
            nn.ReLU()
        )
        self.Encoder8=nn.Sequential(
            #input(512,2,2)
            nn.Conv2d(512,512,2,stride=2),
            nn.InstanceNorm2d(512),
            nn.ReLU()
        )
        self.Decoder8 = nn.Sequential(
            nn.ConvTranspose2d(512,512,2,stride=2),
            nn.InstanceNorm2d(512),
            nn.ReLU()
        )
        # 这里第一次使用反卷积，注意，从低纬向高纬进行反卷积往往不是一一对应的，为了使反卷积和卷积对应起来，通常来讲,padding = (ksize-1)/2 ,output_padding=stride-1
        self.Decoder7 = nn.Sequential(
            nn.ConvTranspose2d(512,512,3,stride=2,padding=1,output_padding=1),
            nn.InstanceNorm2d(512),
            nn.ReLU()
        )
        self.Decoder6 = nn.Sequential(
            nn.ConvTranspose2d(512,512,3,stride=2,padding=1,output_padding=1),
            nn.InstanceNorm2d(512),
            nn.ReLU()
        )
        self.Decoder5=nn.Sequential(
            nn.ConvTranspose2d(512,512,3,stride=2,padding=1,output_padding=1),
            nn.InstanceNorm2d(512),
            nn.ReLU()
        )
        self.Decoder4 = nn.Sequential(
            nn.ConvTranspose2d(512,256,3,stride=2,padding=1,output_padding=1),
            nn.InstanceNorm2d(512),
            nn.ReLU()
        )
        self.Decoder3 = nn.Sequential(
            nn.ConvTranspose2d(256,128,3,stride=2,padding=1,output_padding=1),
            nn.InstanceNorm2d(256),
            nn.ReLU()
        )
        self.Decoder2 = nn.Sequential(
            nn.ConvTranspose2d(128,64,3,stride=2,padding=1,output_padding=1),
            nn.InstanceNorm2d(128),
            nn.ReLU()
        )
        self.Decoder1 = nn.Sequential(
            nn.ConvTranspose2d(64,1,3,stride=2,padding=1,output_padding=1),
            nn.Tanh()
        )
    def forward(self,frame):
        #input(N,3,256,256)
        encoder1 = self.Encoder1(frame)
        encoder2 = self.Encoder2(encoder1)
        encoder3 = self.Encoder3(encoder2)
        encoder4 = self.Encoder4(encoder3)
        encoder5 = self.Encoder5(encoder4)
        encoder6 = self.Encoder6(encoder5)
        encoder7 = self.Encoder7(encoder6)
        encoder8 = self.Encoder8(encoder7)
        
        decoder8 = self.Decoder8(encoder8)+encoder7
        decoder7 = self.Decoder7(decoder8)+encoder6
        decoder6 = self.Decoder6(decoder7)+encoder5
        decoder5 = self.Decoder5(decoder6)+encoder4
        decoder4 = self.Decoder4(decoder5)+encoder3
        decoder3 = self.Decoder3(decoder4)+encoder2
        decoder2 = self.Decoder2(decoder3)+encoder1
        output = self.Decoder1(decoder2)
        return output
        

## 1.2判别模型

In [2]:
class HDRDis(nn.Module):
    def __init__(self):
        super(HDRDis,self).__init__()
        self.Discrim = nn.Sequential(
            # input(4,256,256)
            nn.Conv2d(4,64,3,stride=2,padding=1),
            nn.ReLU(),
            nn.Conv2d(64,128,3,stride=2,padding=1),
            nn.ReLU(),
            nn.Conv2d(128,256,3,stride=2,padding=1),
            nn.ReLU(),
            nn.Conv2d(256,512,3),
            nn.ReLU(),
            nn.Conv2d(512,1,1),
            nn.Sigmoid()
        )
    def forward(self,Events, Gimage):
        frame = torch.cat((Events,Gimage),dim=1)
        # input(N,4,256,256)
        output = self.Discrim(frame)
        return torch.mean(output)

## 1.3 损失函数

In [3]:
class G_loss(nn.Module):
    def __init__(self):
        super(G_loss,self).__init__()
    def forward(self,G,e,g):
        return -torch.mean(torch.log(D(e,G(e))))+torch.mean(torch.abs(g-G(e)))
        
class D_loss(nn.Module):
    def __init__(self):
        super(D_loss,self).__init__()
    def forward(self,D,G,e,g):
        return -torch.log(D(e,g))-torch.log(1-D(e,G(e)))

# 2. 数据处理

In [4]:
from torch.utils.data import Dataset
import numpy as np
import os
import cv2

class Datain(Dataset):
    def __init__(self,InputPath):
        self.InputPath = InputPath
        self.FileList=os.listdir(self.InputPath)
        self.Length = len(self.FileList)
        
    def Float2Image(self,InputFloat):
        Output = 127.5*InputFloat+127.5
        return Output.astype(int)

    def Image2Float(self,InputImage):
        InputImage = InputImage.astype(float)
        return 2*InputImage/255 -1
    
    
    def __getitem__(self,index):
        Image = cv2.imread(os.path.join(self.InputPath,self.FileList[index]))
        Image = cv2.resize(Image,(512,256),interpolation=cv2.INTER_CUBIC)
        Image = self.Image2Float(Image)
        Image = torch.tensor(Image,dtype = torch.float32)
        Input,Output = Image.chunk(2,1)
        Output,_,_ = Output.chunk(3,2)
        return Input.permute(2,0,1),Output.permute(2,0,1)
    def __len__(self):
        return self.Length
    
TrainData = Datain('Sim_EBS_train')
print(TrainData[0][0].shape,TrainData[0][1].shape)

torch.Size([3, 256, 256]) torch.Size([1, 256, 256])


# 3. 实验部分

In [5]:
import os
from torch.utils.data import DataLoader

# 初始化
TrainSetPath = ''
lr=0.0005
BatchSize = 10
Epochs = 100

#指定某块GPU进行训练
os.environ["CUDA_VISIBLE_DEVICES"] = '2'

# 加载数据
TrainLoader=DataLoader(dataset=TrainData,batch_size=BatchSize,shuffle=True)


# 训练模型
G = HDRGen()
D = HDRDis()

G.cuda()
D.cuda()

if os.path.exists('GenModel.pkl'):
    G.load_state_dict(torch.load('GenModel.pkl'))
    print('G model load successfully')
    
if os.path.exists('DisModel.pkl'):
    D.load_state_dict(torch.load('DisModel.pkl'))
    print('D model load successfully') 
    
G_opt = torch.optim.Adam(G.parameters(),lr=lr,betas=(0.9,0.99),weight_decay=0.0005)
D_opt = torch.optim.Adam(D.parameters(),lr=lr,betas=(0.9,0.99),weight_decay=0.0005)
# 设置损失函数
Gloss =G_loss()
Dloss = D_loss()

# 设置学习率下降策略
G_sch = torch.optim.lr_scheduler.StepLR(G_opt, step_size=1, gamma=0.98)  
D_sch = torch.optim.lr_scheduler.StepLR(D_opt, step_size=1, gamma=0.98)  

G.train()
D.train()

for epoch in range(Epochs):
    #更新学习率
    D_sch.step()
    G_sch.step()
    EpochTrainCorrect = 0
    EpochGTrainLoss = 0
    EpochDTrainLoss = 0
    for batch, data in enumerate(TrainLoader):
        DTrainLoss = Dloss(D,G,data[0].cuda(),data[1].cuda())
        D_opt.zero_grad()
        DTrainLoss.backward()
        D_opt.step()
        EpochDTrainLoss+=DTrainLoss
        
        GTrainLoss = Gloss(G,data[0].cuda(),data[1].cuda())
        G_opt.zero_grad()
        GTrainLoss.backward()
        G_opt.step()
        EpochGTrainLoss+=GTrainLoss
        
        print('Batch:{}/{} BatchLoss: G:{:.3f},D:{:.3f}'.format(batch,len(TrainLoader),GTrainLoss,DTrainLoss))
    print('EpochLoss:G:{},D:{}'.format(EpochGTrainLoss,EpochDTrainLoss))
    torch.save(G.state_dict(),'GenModel.pkl')
    torch.save(D.state_dict(),'DisModel.pkl')
    
        

G model load successfully
D model load successfully
Batch:0/88 BatchLoss: G:2.175,D:0.670
Batch:1/88 BatchLoss: G:1.917,D:0.809
Batch:2/88 BatchLoss: G:1.766,D:1.032
Batch:3/88 BatchLoss: G:1.669,D:1.275
Batch:4/88 BatchLoss: G:1.553,D:1.320
Batch:5/88 BatchLoss: G:1.542,D:1.253
Batch:6/88 BatchLoss: G:1.605,D:1.319
Batch:7/88 BatchLoss: G:1.567,D:1.381
Batch:8/88 BatchLoss: G:1.724,D:1.356
Batch:9/88 BatchLoss: G:1.681,D:1.405
Batch:10/88 BatchLoss: G:1.736,D:1.238
Batch:11/88 BatchLoss: G:1.644,D:1.234
Batch:12/88 BatchLoss: G:1.745,D:1.057
Batch:13/88 BatchLoss: G:1.739,D:1.034
Batch:14/88 BatchLoss: G:1.748,D:0.930
Batch:15/88 BatchLoss: G:1.888,D:0.869
Batch:16/88 BatchLoss: G:2.326,D:0.785
Batch:17/88 BatchLoss: G:3.325,D:0.705
Batch:18/88 BatchLoss: G:3.476,D:0.648
Batch:19/88 BatchLoss: G:3.095,D:0.574
Batch:20/88 BatchLoss: G:2.778,D:0.479
Batch:21/88 BatchLoss: G:2.580,D:0.433
Batch:22/88 BatchLoss: G:2.427,D:0.396
Batch:23/88 BatchLoss: G:2.502,D:0.360
Batch:24/88 BatchLoss:

Batch:31/88 BatchLoss: G:2.569,D:0.519
Batch:32/88 BatchLoss: G:2.503,D:0.623
Batch:33/88 BatchLoss: G:2.731,D:0.413
Batch:34/88 BatchLoss: G:2.444,D:0.534
Batch:35/88 BatchLoss: G:1.932,D:0.468
Batch:36/88 BatchLoss: G:2.341,D:0.434
Batch:37/88 BatchLoss: G:3.062,D:0.437
Batch:38/88 BatchLoss: G:3.358,D:0.506
Batch:39/88 BatchLoss: G:2.558,D:0.442
Batch:40/88 BatchLoss: G:2.414,D:0.513
Batch:41/88 BatchLoss: G:2.554,D:0.600
Batch:42/88 BatchLoss: G:2.927,D:0.519
Batch:43/88 BatchLoss: G:2.160,D:0.440
Batch:44/88 BatchLoss: G:2.143,D:0.452
Batch:45/88 BatchLoss: G:2.645,D:0.418
Batch:46/88 BatchLoss: G:2.792,D:0.589
Batch:47/88 BatchLoss: G:3.055,D:0.574
Batch:48/88 BatchLoss: G:3.060,D:0.590
Batch:49/88 BatchLoss: G:2.916,D:0.530
Batch:50/88 BatchLoss: G:2.617,D:0.502
Batch:51/88 BatchLoss: G:1.820,D:0.587
Batch:52/88 BatchLoss: G:1.891,D:0.524
Batch:53/88 BatchLoss: G:2.260,D:0.463
Batch:54/88 BatchLoss: G:2.438,D:0.528
Batch:55/88 BatchLoss: G:2.593,D:0.504
Batch:56/88 BatchLoss: G:

Batch:64/88 BatchLoss: G:1.761,D:0.812
Batch:65/88 BatchLoss: G:1.661,D:0.866
Batch:66/88 BatchLoss: G:1.630,D:0.829
Batch:67/88 BatchLoss: G:1.616,D:0.832
Batch:68/88 BatchLoss: G:1.658,D:0.762
Batch:69/88 BatchLoss: G:1.753,D:0.767
Batch:70/88 BatchLoss: G:2.618,D:0.676
Batch:71/88 BatchLoss: G:2.842,D:0.651
Batch:72/88 BatchLoss: G:2.462,D:0.755
Batch:73/88 BatchLoss: G:2.176,D:0.636
Batch:74/88 BatchLoss: G:2.013,D:0.689
Batch:75/88 BatchLoss: G:1.964,D:0.776
Batch:76/88 BatchLoss: G:2.295,D:0.716
Batch:77/88 BatchLoss: G:2.850,D:0.689
Batch:78/88 BatchLoss: G:3.235,D:0.770
Batch:79/88 BatchLoss: G:2.910,D:0.793
Batch:80/88 BatchLoss: G:2.295,D:0.821
Batch:81/88 BatchLoss: G:2.199,D:0.615
Batch:82/88 BatchLoss: G:2.423,D:0.582
Batch:83/88 BatchLoss: G:2.290,D:0.491
Batch:84/88 BatchLoss: G:2.154,D:0.499
Batch:85/88 BatchLoss: G:2.017,D:0.612
Batch:86/88 BatchLoss: G:2.021,D:0.649
Batch:87/88 BatchLoss: G:2.099,D:0.639
EpochLoss:G:191.28846740722656,D:59.356563568115234
Batch:0/88 B

Batch:7/88 BatchLoss: G:1.578,D:0.955
Batch:8/88 BatchLoss: G:1.421,D:1.014
Batch:9/88 BatchLoss: G:1.418,D:1.094
Batch:10/88 BatchLoss: G:1.341,D:1.009
Batch:11/88 BatchLoss: G:1.239,D:1.113
Batch:12/88 BatchLoss: G:1.290,D:1.064
Batch:13/88 BatchLoss: G:1.263,D:1.165
Batch:14/88 BatchLoss: G:1.254,D:1.193
Batch:15/88 BatchLoss: G:1.239,D:1.251
Batch:16/88 BatchLoss: G:1.255,D:1.231
Batch:17/88 BatchLoss: G:1.259,D:1.201
Batch:18/88 BatchLoss: G:1.182,D:1.232
Batch:19/88 BatchLoss: G:1.231,D:1.230
Batch:20/88 BatchLoss: G:1.214,D:1.262
Batch:21/88 BatchLoss: G:1.208,D:1.253
Batch:22/88 BatchLoss: G:1.174,D:1.308
Batch:23/88 BatchLoss: G:1.202,D:1.250
Batch:24/88 BatchLoss: G:1.252,D:1.288
Batch:25/88 BatchLoss: G:1.230,D:1.284
Batch:26/88 BatchLoss: G:1.283,D:1.203
Batch:27/88 BatchLoss: G:1.165,D:1.291
Batch:28/88 BatchLoss: G:1.364,D:1.342
Batch:29/88 BatchLoss: G:1.272,D:1.319
Batch:30/88 BatchLoss: G:1.363,D:1.332
Batch:31/88 BatchLoss: G:1.343,D:1.326
Batch:32/88 BatchLoss: G:1.3

Batch:40/88 BatchLoss: G:0.794,D:2.027
Batch:41/88 BatchLoss: G:0.806,D:2.080
Batch:42/88 BatchLoss: G:0.793,D:2.133
Batch:43/88 BatchLoss: G:0.801,D:2.233
Batch:44/88 BatchLoss: G:0.799,D:2.155
Batch:45/88 BatchLoss: G:0.741,D:2.138
Batch:46/88 BatchLoss: G:0.759,D:2.164
Batch:47/88 BatchLoss: G:0.814,D:2.266
Batch:48/88 BatchLoss: G:0.794,D:2.255
Batch:49/88 BatchLoss: G:0.785,D:2.327
Batch:50/88 BatchLoss: G:0.822,D:2.329
Batch:51/88 BatchLoss: G:0.895,D:2.436
Batch:52/88 BatchLoss: G:0.942,D:2.401
Batch:53/88 BatchLoss: G:0.909,D:2.443
Batch:54/88 BatchLoss: G:0.992,D:2.315
Batch:55/88 BatchLoss: G:1.326,D:2.333
Batch:56/88 BatchLoss: G:2.564,D:2.195
Batch:57/88 BatchLoss: G:1.543,D:3.192
Batch:58/88 BatchLoss: G:1.058,D:2.362
Batch:59/88 BatchLoss: G:0.976,D:2.574
Batch:60/88 BatchLoss: G:0.949,D:2.983
Batch:61/88 BatchLoss: G:0.888,D:2.663
Batch:62/88 BatchLoss: G:0.910,D:2.701
Batch:63/88 BatchLoss: G:0.842,D:2.679
Batch:64/88 BatchLoss: G:0.863,D:2.688
Batch:65/88 BatchLoss: G:

...

Batch:79/88 BatchLoss: G:0.880,D:1.387
Batch:80/88 BatchLoss: G:0.832,D:1.381
Batch:81/88 BatchLoss: G:0.829,D:1.378
Batch:82/88 BatchLoss: G:0.814,D:1.377
Batch:83/88 BatchLoss: G:0.832,D:1.379
Batch:84/88 BatchLoss: G:0.852,D:1.380
Batch:85/88 BatchLoss: G:0.839,D:1.382
Batch:86/88 BatchLoss: G:0.895,D:1.374
Batch:87/88 BatchLoss: G:0.916,D:1.420
EpochLoss:G:74.7298583984375,D:121.90226745605469
Batch:0/88 BatchLoss: G:0.781,D:1.384
Batch:1/88 BatchLoss: G:0.764,D:1.388
Batch:2/88 BatchLoss: G:0.867,D:1.383
Batch:3/88 BatchLoss: G:0.967,D:1.359
Batch:4/88 BatchLoss: G:0.915,D:1.393
Batch:5/88 BatchLoss: G:0.857,D:1.397
Batch:6/88 BatchLoss: G:0.759,D:1.351
Batch:7/88 BatchLoss: G:0.761,D:1.365
Batch:8/88 BatchLoss: G:0.798,D:1.393
Batch:9/88 BatchLoss: G:0.863,D:1.393
Batch:10/88 BatchLoss: G:0.938,D:1.377
Batch:11/88 BatchLoss: G:0.874,D:1.385
Batch:12/88 BatchLoss: G:0.798,D:1.376
Batch:13/88 BatchLoss: G:0.769,D:1.390
Batch:14/88 BatchLoss: G:0.789,D:1.392
Batch:15/88 BatchLoss: G

Batch:22/88 BatchLoss: G:0.822,D:1.394
Batch:23/88 BatchLoss: G:0.863,D:1.392
Batch:24/88 BatchLoss: G:0.950,D:1.412
Batch:25/88 BatchLoss: G:0.875,D:1.399
Batch:26/88 BatchLoss: G:0.880,D:1.410
Batch:27/88 BatchLoss: G:0.801,D:1.394
Batch:28/88 BatchLoss: G:0.841,D:1.397
Batch:29/88 BatchLoss: G:0.714,D:1.400
Batch:30/88 BatchLoss: G:0.818,D:1.390
Batch:31/88 BatchLoss: G:0.721,D:1.395
Batch:32/88 BatchLoss: G:0.757,D:1.390
Batch:33/88 BatchLoss: G:0.809,D:1.388
Batch:34/88 BatchLoss: G:0.797,D:1.390
Batch:35/88 BatchLoss: G:0.803,D:1.387
Batch:36/88 BatchLoss: G:0.828,D:1.389
Batch:37/88 BatchLoss: G:0.823,D:1.379
Batch:38/88 BatchLoss: G:0.852,D:1.390
Batch:39/88 BatchLoss: G:0.911,D:1.398
Batch:40/88 BatchLoss: G:0.875,D:1.395
Batch:41/88 BatchLoss: G:0.805,D:1.383
Batch:42/88 BatchLoss: G:0.796,D:1.375
Batch:43/88 BatchLoss: G:0.795,D:1.389
Batch:44/88 BatchLoss: G:0.831,D:1.393
Batch:45/88 BatchLoss: G:0.780,D:1.390
Batch:46/88 BatchLoss: G:0.745,D:1.393
Batch:47/88 BatchLoss: G:

Batch:54/88 BatchLoss: G:0.765,D:1.390
Batch:55/88 BatchLoss: G:0.777,D:1.392
Batch:56/88 BatchLoss: G:0.825,D:1.391
Batch:57/88 BatchLoss: G:0.834,D:1.389
Batch:58/88 BatchLoss: G:0.884,D:1.394
Batch:59/88 BatchLoss: G:0.859,D:1.392
Batch:60/88 BatchLoss: G:0.763,D:1.380
Batch:61/88 BatchLoss: G:0.731,D:1.383
Batch:62/88 BatchLoss: G:0.785,D:1.379
Batch:63/88 BatchLoss: G:0.774,D:1.385
Batch:64/88 BatchLoss: G:0.820,D:1.385
Batch:65/88 BatchLoss: G:0.900,D:1.386
Batch:66/88 BatchLoss: G:0.851,D:1.383
Batch:67/88 BatchLoss: G:0.816,D:1.385
Batch:68/88 BatchLoss: G:0.789,D:1.382
Batch:69/88 BatchLoss: G:0.759,D:1.389
Batch:70/88 BatchLoss: G:0.791,D:1.387
Batch:71/88 BatchLoss: G:0.860,D:1.392
Batch:72/88 BatchLoss: G:0.814,D:1.388
Batch:73/88 BatchLoss: G:0.808,D:1.390
Batch:74/88 BatchLoss: G:0.814,D:1.386
Batch:75/88 BatchLoss: G:0.829,D:1.392
Batch:76/88 BatchLoss: G:0.817,D:1.392
Batch:77/88 BatchLoss: G:0.762,D:1.390
Batch:78/88 BatchLoss: G:0.825,D:1.389
Batch:79/88 BatchLoss: G:

Batch:87/88 BatchLoss: G:0.788,D:1.384
EpochLoss:G:70.32579803466797,D:121.70934295654297
Batch:0/88 BatchLoss: G:0.802,D:1.383
Batch:1/88 BatchLoss: G:0.796,D:1.384
Batch:2/88 BatchLoss: G:0.831,D:1.389
Batch:3/88 BatchLoss: G:0.816,D:1.389
Batch:4/88 BatchLoss: G:0.802,D:1.385
Batch:5/88 BatchLoss: G:0.794,D:1.383
Batch:6/88 BatchLoss: G:0.787,D:1.380
Batch:7/88 BatchLoss: G:0.791,D:1.384
Batch:8/88 BatchLoss: G:0.811,D:1.381
Batch:9/88 BatchLoss: G:0.822,D:1.382
Batch:10/88 BatchLoss: G:0.784,D:1.378
Batch:11/88 BatchLoss: G:0.772,D:1.381
Batch:12/88 BatchLoss: G:0.770,D:1.381
Batch:13/88 BatchLoss: G:0.790,D:1.381
Batch:14/88 BatchLoss: G:0.813,D:1.380
Batch:15/88 BatchLoss: G:0.815,D:1.379
Batch:16/88 BatchLoss: G:0.793,D:1.377
Batch:17/88 BatchLoss: G:0.775,D:1.378
Batch:18/88 BatchLoss: G:0.800,D:1.382
Batch:19/88 BatchLoss: G:0.816,D:1.381
Batch:20/88 BatchLoss: G:0.813,D:1.384
Batch:21/88 BatchLoss: G:0.814,D:1.382
Batch:22/88 BatchLoss: G:0.786,D:1.380
Batch:23/88 BatchLoss: 

KeyboardInterrupt: 